In [1]:
import mido
import os
import numpy as np
import sys

In [2]:
def eliminate_midifile(file):
    has_drums = False
    has_piano = False
    has_guitar = False
    has_bass = False
    i_drums = []
    i_piano = []
    i_guitar = []
    i_bass = []
    
    piano_range = range(0,8)
    guitar_range = range(24,32)
    bass_range = range(32,40)
    
    #short conditions:
    for i,track in enumerate(file.tracks):
        names= track.name.split()
        for name in names:
            if name.lower()=='drums':
                has_drums=True
                i_drums.append(i)
                continue
            elif name.lower()=='piano':
                has_piano=True
                i_piano.append(i)
                continue
            elif name.lower()=='guitar':
                has_guitar=True
                i_guitar.append(i)
                continue
            elif name.lower()=='bass':
                has_bass=True
                i_bass.append(i)
                continue
    #testing if all tracks dont have the song name as their track name
    l=len(file.tracks)
    if len(i_drums)==l or len(i_piano)==l or len(i_guitar)==l or len(i_bass)==l:
        has_drums,has_piano,has_guitar,has_bass=False,False,False,False
        # if that's the case we can't be sure that track names correspond to the instruments,
        # so it's better to just scrap it and go along with the long conditions
        
    #long_conditions
    for track in file.tracks:
        for msg in track[0:20]: #first 20 msgs
            #condition 1 : 4/4 time signature on tempo data byte
            if msg.bytes()[:2]==[255,88]:
                if not(msg.bytes()[2:4]==[4,4]):
                    return False
            #condition 2 : drums 
            if has_drums==False:
                if msg.is_meta==False:
                    if msg.type()=='program_change':
                        if msg.dict()['channel']==9:
                            has_drums=True
            #condition 3 : piano
            if has_piano==False:
                if msg.is_meta==False:
                    if msg.type()=='program_change':
                        if msg.dict()['program'] in piano_range:
                            has_piano=True
            #condition 4 : guitar
            if has_guitar==False:
                if msg.is_meta==False:
                    if msg.type()=='program_change':
                        if msg.dict()['program'] in guitar_range:
                            has_guitar=True
            #condition 5 : bass
            if has_bass==False:
                 if msg.is_meta==False:
                    if msg.type()=='program_change':
                        if msg.dict()['program'] in bass_range:
                            has_bass=True
    #endloop
    if has_drums and has_piano and has_guitar and has_bass:
        return True
    else:
        return False
        

In [3]:
def array_on_track(track,ticks):
    # unlike the first drums program where i had lists,
    #i will work here with arrays to make it more efficient
    
    first_array = np.zeros(shape=(len(track),3))
    time = 0 #ticks
    for i,msg in enumerate(track):
        time+=msg.time
        if msg.type=='control_change' or msg.type=='program_change' or msg.type=='note_off':
            continue
        elif not msg.is_meta: # we only want note_on messages
            first_array[i,0]=1
            first_array[i,1]=msg.bytes()[1] #note_value 0..127
            # we scrap velocity
            first_array[i,2]=round(time*4/ticks) #quantized time in ticks
    # we have now an event-based list of notes
    # let's transform it into a time series with note value on the second dimension
    
    second_array = np.zeros(shape=(round(time*4/ticks),129))
    for note in first_array:
        if note[0]==1:
            second_array[int(note[2]),int(note[1])]=1
            second_array[int(note[2]),128]=1
    # in doing so, we skipped the data regarding velocity and note_off events
    # but, as this is just the input for the model and won't be used to create midifiles,
    # we can allow ourselves to simplify the data 
    return second_array
        
  

In [4]:
def drum_array_on_track(track,ticks):
    first_array = np.zeros(shape=(len(track),5))
    time = 0 #ticks
    
    #converting standard drum notation to indexes 0-8
    gm_to_ix ={
        35:0,36:0,
        37:1,38:1,39:1,40:1,
        42:2,44:2,
        46:3,
        41:4,43:4,
        45:5,47:5,
        48:6,50:6,
        51:7,53:7,59:7,
        49:8,52:8,55:8,57:8
    }
    #kick,snare,HHc,HHo,tom3,tom2,tom1,ride,crash
    #0   ,1    ,2  ,3  ,4   ,5   ,6   ,7   ,8
    
    for i,msg in enumerate(track):
        time+=msg.time
        if msg.type=='control_change' or msg.type=='program_change' or msg.type=='note_off':
            continue
        elif not msg.is_meta:
            if msg.bytes()[1] in gm_to_ix.keys(): # to prevent key errors
                first_array[i,0]=1 #note or no note 
                first_array[i,1]=gm_to_ix[msg.bytes()[1]] #note_value 0-8
                first_array[i,2]=msg.bytes()[2]/127 #velocity : float 0..1
                first_array[i,3]=round(time*4/ticks) #quantized time in ticks
                first_array[i,4]=(time*4/ticks)-round(time*4/ticks) #offset in ticks
    #####
    #let's create the time series array:
    second_array=np.zeros((round(time*4/ticks),28))
    
    for note in first_array:
        if note[0]==1:
            second_array[int(note[3]),int(note[1])]=1 # note
            second_array[int(note[3]),9+int(note[1])]=note[2] # vel
            second_array[int(note[3]),18+int(note[1])]=note[4] # time offset
            second_array[int(note[3]),27]=1 #tells us if a note is played at said timestep (useful later)
            # we do it this way to avoid needing multi-dimensional arrays.
            # this will probably be useful in the next step
            # when we'll try to find a window where all instruments are playing
    return second_array 

In [5]:
def check_window(array1,array2,array3,array4,barsize):
    #checking each 128x1 sized array
    for array in [array1,array2,array3,array4]:
        counter=0
        for val in array:
            if val==0: #note not found
                counter+=1
                if counter>2*barsize: 
                    return False
            else: #note found
                counter=0
            
    else:
        return True

    #here we're checking that there's not more than 2bars of silence for each instrument

In [6]:
def create_arrays(file):
    ticks = file.ticks_per_beat
    
    
    nb_bass=0
    #nb_melody=0 #let's split it in guitar and piano and have 1 instrument max per class
    nb_guitar=0
    nb_piano=0
    #nb_drums=1  # we don't care if there are more instruments for percussion,
    # because we only select those for drums (ctrl nb : ca 35-59)
    
    piano_ix=[]
    guitar_ix=[]
    bass_ix=[]
    
    i_drums=0
    
    piano_range = range(0,8)
    guitar_range = range(24,32)
    bass_range = range(32,40)
    
    ##### filtering tracks:
    for i,track in enumerate(file.tracks):
        for msg in track[0:20]:
            if msg.is_meta == False:
                if msg.type=='program_change':
                    if msg.dict()['channel']==9:
                        i_drums=i
                    elif msg.dict()['program'] in piano_range:
                        nb_piano+=1
                        piano_ix.append(i)
                    elif msg.dict()['program'] in guitar_range:
                        nb_guitar+=1
                        guitar_ix.append(i)
                    elif msg.dict()['program'] in bass_range:
                        nb_bass+=1
                        bass_ix.append(i)
    #####
    created=False
    drum_array1 = drum_array_on_track(file.tracks[i_drums],ticks)
    for i_piano in piano_ix:
        for i_guitar in guitar_ix:
            for i_bass in bass_ix:
                #this loops over all possibilities for these instruments 
                piano_array1 = array_on_track(file.tracks[int(i_piano)],ticks)
                guitar_array1 = array_on_track(file.tracks[i_guitar],ticks)
                bass_array1 = array_on_track(file.tracks[i_bass],ticks)
                
                #######
                # now for the sliding window part
                max_length = min([
                    drum_array1.shape[0],
                    piano_array1.shape[0],
                    guitar_array1.shape[0],
                    bass_array1.shape[0]])
                #let's settle on 8 bars for our sliding window
                bar_size=16 #events per bar
                ws=128 # window_size = 8*16 # variable name was too long otherwise
                stride=32
                max_usable_length = max_length-ws
                
                for p in range(0,max_usable_length,stride):
                    #checking if there are all instruments in this selected window:
                    if check_window(
                        drum_array1[p:p+ws,27],
                        piano_array1[p:p+ws,128],
                        guitar_array1[p:p+ws,128],
                        bass_array1[p:p+ws,128],
                        bar_size
                    )==True:
                        
                        # adding to final arrays if already created
                        if created==False:
                            created=True
                            drum_final=drum_array1[p:p+ws,0:27].copy().reshape(1,ws,27)
                            piano_final=piano_array1[p:p+ws,0:128].copy().reshape(1,ws,128)
                            guitar_final=guitar_array1[p:p+ws,0:128].copy().reshape(1,ws,128)
                            bass_final=bass_array1[p:p+ws,0:128].copy().reshape(1,ws,128)
                        else:
                            drum_final=np.concatenate([
                                drum_final,
                                drum_array1[p:p+ws,0:27].copy().reshape(1,ws,27)],axis=0)
                            piano_final=np.concatenate([
                                piano_final,
                                piano_array1[p:p+ws,0:128].copy().reshape(1,ws,128)],axis=0)
                            guitar_final=np.concatenate([
                                guitar_final,
                                guitar_array1[p:p+ws,0:128].copy().reshape(1,ws,128)],axis=0)
                            bass_final=np.concatenate([
                                bass_final,
                                bass_array1[p:p+ws,0:128].copy().reshape(1,ws,128)],axis=0)
                    
    if created==True:
        return [drum_final,piano_final,guitar_final,bass_final]
    else:
        return False

In [7]:
with open('dir_keywords.txt') as f:
      lines = f.readlines()
prev_dirs=[]
for line in lines:
    prev_dirs.append(line.strip('\n'))

In [ ]:
"""loaded = np.load('datasets/TM_dataset_enc9_part2.npz') 

array_drum = loaded['array1_drm']
array_piano = loaded['array2_pia']
array_gtr = loaded['array3_gtr']
array_bass = loaded['array4_bss']"""

In [8]:
current_dirs=[]
created = False
path='datasets/TM_dataset_enc9_part5'
i=0
try:
    for dirpath,dirnames,files in os.walk('datasets/lmd_matched'):
        if dirpath[21:24] in prev_dirs:
            continue
        elif dirpath[21:24] not in current_dirs:
                current_dirs.append(dirpath[21:24])
        print(f'\tDirectory : {dirpath}')
        with os.scandir(dirpath) as entries:
            for entry in entries:
                x,extension = os.path.splitext(entry)
                if extension=='.mid':
                    #-----------------------------------------
                    if i%100==0:
                        print(f'file {i+1} out of 116,189')
                    if i%1000==0 and i!=0:
                        print('\t\t saving dataset on disk...')
                        if created:
                            np.savez_compressed(
                                path,
                                array1_drm=array_drum,
                                array2_pia=array_piano,
                                array3_gtr=array_gtr,
                                array4_bss=array_bass)
                        print('\t\t saving current_dirs on hist file...')
                        with open('dir_keywords.txt','a') as f:
                            for k in current_dirs:
                                f.write(k+'\n')
                        current_dirs.clear()
                    #-----------------------------------------
                    try:
                        file = mido.MidiFile(entry)
                        if not created:
                            created = True
                            try:
                                array_drum,array_piano,array_gtr,array_bass = create_arrays(file)
                            except KeyboardInterrupt:
                                raise KeyboardInterrupt
                            except:
                                created = False
                        else:
                            try:
                                array1,array2,array3,array4 = create_arrays(file)
                                array_drum=np.concatenate([array_drum,array1],axis=0)
                                array_piano=np.concatenate([array_piano,array2],axis=0)
                                array_gtr=np.concatenate([array_gtr,array3],axis=0)
                                array_bass=np.concatenate([array_bass,array4],axis=0)
                            except KeyboardInterrupt:
                                raise KeyboardInterrupt
                            except:
                                pass
                    except KeyboardInterrupt:
                        raise KeyboardInterrupt  
                    except:
                        print(f'\tfile {i+1} not valid, skipping..')
                    i+=1
except:
    print('\n',20*'-')
    print('\tError: ',sys.exc_info()[0])
    print('number of files read : ',i+1)
else:
    print('\n','end')
    print('number of files read : ',i+1)

	Directory : datasets/lmd_matched/K/S
	Directory : datasets/lmd_matched/K/S/R
	Directory : datasets/lmd_matched/K/S/R/TRKSRVG128F428526C
file 1 out of 116,189
	Directory : datasets/lmd_matched/K/S/U
	Directory : datasets/lmd_matched/K/S/U/TRKSUEH128E0787846
	Directory : datasets/lmd_matched/K/S/U/TRKSUGL128F9303DAE
	Directory : datasets/lmd_matched/K/S/U/TRKSUBO128F427CB79
	Directory : datasets/lmd_matched/K/S/U/TRKSUFV128F4265476
	Directory : datasets/lmd_matched/K/S/I
	Directory : datasets/lmd_matched/K/S/I/TRKSIVV12903D0739F
	Directory : datasets/lmd_matched/K/S/N
	Directory : datasets/lmd_matched/K/S/N/TRKSNOY128F9307A61
	Directory : datasets/lmd_matched/K/S/N/TRKSNFI128F930AA8F
	Directory : datasets/lmd_matched/K/S/G
	Directory : datasets/lmd_matched/K/S/G/TRKSGLT128F14618B1
	Directory : datasets/lmd_matched/K/S/G/TRKSGTO12903CF44D0
	Directory : datasets/lmd_matched/K/S/T
	Directory : datasets/lmd_matched/K/S/T/TRKSTOL128F92DA76D
	Directory : datasets/lmd_matched/K/S/S
	Directory 

	Directory : datasets/lmd_matched/K/F/T
	Directory : datasets/lmd_matched/K/F/T/TRKFTQA128F93067BC
	Directory : datasets/lmd_matched/K/F/F
	Directory : datasets/lmd_matched/K/F/F/TRKFFFU128F4253B70
	Directory : datasets/lmd_matched/K/F/F/TRKFFMV128F425A5B2
	Directory : datasets/lmd_matched/K/F/O
	Directory : datasets/lmd_matched/K/F/O/TRKFONV12903CF767C
	Directory : datasets/lmd_matched/K/F/O/TRKFOBM128F92DCCEB
	Directory : datasets/lmd_matched/K/F/O/TRKFOUP128E0786014
	Directory : datasets/lmd_matched/K/F/H
	Directory : datasets/lmd_matched/K/F/H/TRKFHVX128F92DDD88
	Directory : datasets/lmd_matched/K/F/H/TRKFHLP128F149A820
	Directory : datasets/lmd_matched/K/F/H/TRKFHJE128F4238A1F
	Directory : datasets/lmd_matched/K/F/J
	Directory : datasets/lmd_matched/K/F/J/TRKFJQL128F4289A6B
	Directory : datasets/lmd_matched/K/F/J/TRKFJTC12903CFAC3A
	Directory : datasets/lmd_matched/K/F/J/TRKFJCW128F148F467
	Directory : datasets/lmd_matched/K/F/J/TRKFJWS128F931227F
	Directory : datasets/lmd_matched

	Directory : datasets/lmd_matched/K/H/W
	Directory : datasets/lmd_matched/K/H/W/TRKHWEC128F429F793
	Directory : datasets/lmd_matched/K/H/W/TRKHWXL128F42782F2
	Directory : datasets/lmd_matched/K/M
	Directory : datasets/lmd_matched/K/M/R
	Directory : datasets/lmd_matched/K/M/R/TRKMROX128F42581D6
	Directory : datasets/lmd_matched/K/M/R/TRKMRQI128F42A00AF
	Directory : datasets/lmd_matched/K/M/R/TRKMRPC12903CE34E5
	Directory : datasets/lmd_matched/K/M/R/TRKMRBI128F92CC2BD
	Directory : datasets/lmd_matched/K/M/R/TRKMRRT128F931F15C
	Directory : datasets/lmd_matched/K/M/U
	Directory : datasets/lmd_matched/K/M/U/TRKMUBL128F42943A5
	Directory : datasets/lmd_matched/K/M/U/TRKMUPC128F1453AEA
	Directory : datasets/lmd_matched/K/M/N
	Directory : datasets/lmd_matched/K/M/N/TRKMNEL128F425A058
	Directory : datasets/lmd_matched/K/M/G
	Directory : datasets/lmd_matched/K/M/G/TRKMGOC128F92F6589
	Directory : datasets/lmd_matched/K/M/A
	Directory : datasets/lmd_matched/K/M/A/TRKMALO128F93211BF
	Directory : d

	Directory : datasets/lmd_matched/K/C/R/TRKCRLT128F42A58DE
	Directory : datasets/lmd_matched/K/C/U
	Directory : datasets/lmd_matched/K/C/U/TRKCUGH128F429080D
	Directory : datasets/lmd_matched/K/C/U/TRKCUGS128F42A15B6
	Directory : datasets/lmd_matched/K/C/U/TRKCUKX128F92DEA65
	Directory : datasets/lmd_matched/K/C/N
	Directory : datasets/lmd_matched/K/C/N/TRKCNUS128F4267B31
	Directory : datasets/lmd_matched/K/C/G
	Directory : datasets/lmd_matched/K/C/G/TRKCGZO128F9302C91
	Directory : datasets/lmd_matched/K/C/Z
	Directory : datasets/lmd_matched/K/C/Z/TRKCZYQ128F426B019
	Directory : datasets/lmd_matched/K/C/Z/TRKCZIZ128F92FD7DA
	Directory : datasets/lmd_matched/K/C/Z/TRKCZOK128E07843EA
	Directory : datasets/lmd_matched/K/C/T
	Directory : datasets/lmd_matched/K/C/T/TRKCTXP128F92F74ED
	Directory : datasets/lmd_matched/K/C/T/TRKCTNX12903CF21F3
	Directory : datasets/lmd_matched/K/C/S
	Directory : datasets/lmd_matched/K/C/S/TRKCSJE128F422DFBA
	Directory : datasets/lmd_matched/K/C/S/TRKCSCW128F4

	Directory : datasets/lmd_matched/K/V/F/TRKVFWW128F42376B5
	Directory : datasets/lmd_matched/K/V/O
	Directory : datasets/lmd_matched/K/V/O/TRKVOCW128F931FD8F
	Directory : datasets/lmd_matched/K/V/O/TRKVOWX128F930D730
	Directory : datasets/lmd_matched/K/V/H
	Directory : datasets/lmd_matched/K/V/H/TRKVHKB128F92EDA35
	Directory : datasets/lmd_matched/K/V/H/TRKVHZE128E078E7FB
file 1401 out of 116,189
	Directory : datasets/lmd_matched/K/V/M
	Directory : datasets/lmd_matched/K/V/M/TRKVMTN128F42B0D4E
	Directory : datasets/lmd_matched/K/V/J
	Directory : datasets/lmd_matched/K/V/J/TRKVJDV128F4253B30
	Directory : datasets/lmd_matched/K/V/D
	Directory : datasets/lmd_matched/K/V/D/TRKVDSB12903CACCC4
	Directory : datasets/lmd_matched/K/V/D/TRKVDIF128F92C4156
	Directory : datasets/lmd_matched/K/V/D/TRKVDNI128F145F4D3
	Directory : datasets/lmd_matched/K/V/V
	Directory : datasets/lmd_matched/K/V/V/TRKVVMC128F4252EB2
	Directory : datasets/lmd_matched/K/V/V/TRKVVVK12903CB592C
	Directory : datasets/lmd_m

...

	Directory : datasets/lmd_matched/W/V/V
	Directory : datasets/lmd_matched/W/V/V/TRWVVMX128F1460888
	Directory : datasets/lmd_matched/W/V/V/TRWVVFF128F92CFA0E
	Directory : datasets/lmd_matched/W/V/V/TRWVVCC128F422A966
	Directory : datasets/lmd_matched/W/V/V/TRWVVWI128F4238412
	Directory : datasets/lmd_matched/W/V/Q
	Directory : datasets/lmd_matched/W/V/Q/TRWVQLD128F147DB22
	Directory : datasets/lmd_matched/W/V/X
	Directory : datasets/lmd_matched/W/V/X/TRWVXJG128F93376A0
	Directory : datasets/lmd_matched/W/V/X/TRWVXGK128F149BD11
	Directory : datasets/lmd_matched/W/V/X/TRWVXJB128F930D543
	Directory : datasets/lmd_matched/W/V/E
	Directory : datasets/lmd_matched/W/V/E/TRWVEWE128F4267A96
	Directory : datasets/lmd_matched/W/V/B
	Directory : datasets/lmd_matched/W/V/B/TRWVBZK128E0794B40
	Directory : datasets/lmd_matched/W/V/K
	Directory : datasets/lmd_matched/W/V/K/TRWVKLX12903D0A0FB
	Directory : datasets/lmd_matched/W/V/K/TRWVKAJ128F92F123B
	Directory : datasets/lmd_matched/W/V/L
	Directory :

	Directory : datasets/lmd_matched/W/X/D/TRWXDJH128F4280AF7
	Directory : datasets/lmd_matched/W/X/D/TRWXDHA128EF35A607
	file 19361 not valid, skipping..
	Directory : datasets/lmd_matched/W/X/D/TRWXDZF128F92F59A3
	file 19372 not valid, skipping..
	Directory : datasets/lmd_matched/W/X/D/TRWXDCO128F4294EBC
	Directory : datasets/lmd_matched/W/X/Q
	Directory : datasets/lmd_matched/W/X/Q/TRWXQZX128F9318EB8
	Directory : datasets/lmd_matched/W/X/Q/TRWXQTQ128F42507C1
	Directory : datasets/lmd_matched/W/X/Q/TRWXQQK128E078FC93
	Directory : datasets/lmd_matched/W/X/Q/TRWXQJZ128F9337E76
file 19401 out of 116,189
	Directory : datasets/lmd_matched/W/X/Q/TRWXQSI128F4261083
	Directory : datasets/lmd_matched/W/X/X
	Directory : datasets/lmd_matched/W/X/X/TRWXXNY128F932154D
	Directory : datasets/lmd_matched/W/X/X/TRWXXVA128F1453470
	Directory : datasets/lmd_matched/W/X/E
	Directory : datasets/lmd_matched/W/X/E/TRWXEQA128F4290B5E
	Directory : datasets/lmd_matched/W/X/E/TRWXEHB128F931C5A7
	Directory : datase

	Directory : datasets/lmd_matched/W/B/D
	Directory : datasets/lmd_matched/W/B/D/TRWBDPG128F4238465
	Directory : datasets/lmd_matched/W/B/D/TRWBDFK12903D110E9
	Directory : datasets/lmd_matched/W/B/D/TRWBDAW128F426BD99
	Directory : datasets/lmd_matched/W/B/D/TRWBDTO12903CE9834
	Directory : datasets/lmd_matched/W/B/V
	Directory : datasets/lmd_matched/W/B/V/TRWBVNF128F92F35E1
	Directory : datasets/lmd_matched/W/B/V/TRWBVKH12903CF0426
	Directory : datasets/lmd_matched/W/B/V/TRWBVNO128F92FB9DC
	Directory : datasets/lmd_matched/W/B/V/TRWBVBZ12903CCAA0A
	Directory : datasets/lmd_matched/W/B/V/TRWBVRS128F4230236
	Directory : datasets/lmd_matched/W/B/Q
	Directory : datasets/lmd_matched/W/B/Q/TRWBQLZ128F1464396
	Directory : datasets/lmd_matched/W/B/Q/TRWBQYC128F426BDEE
	Directory : datasets/lmd_matched/W/B/E
	Directory : datasets/lmd_matched/W/B/E/TRWBEUO128F421ACA6
	Directory : datasets/lmd_matched/W/B/E/TRWBELL128F92F540D
file 19801 out of 116,189
	Directory : datasets/lmd_matched/W/B/E/TRWBEVD

	Directory : datasets/lmd_matched/W/L/B/TRWLBMD128EF3663C8
	Directory : datasets/lmd_matched/W/L/B/TRWLBLZ12903CE895E
	Directory : datasets/lmd_matched/W/L/K
	Directory : datasets/lmd_matched/W/L/K/TRWLKYP128F92FD34D
	Directory : datasets/lmd_matched/W/L/K/TRWLKKD128F4287B02
	Directory : datasets/lmd_matched/W/L/K/TRWLKLI128F42307B4
	Directory : datasets/lmd_matched/W/L/K/TRWLKID128F92F080D
	Directory : datasets/lmd_matched/W/L/K/TRWLKNU128F930778F
	Directory : datasets/lmd_matched/W/L/K/TRWLKDG128F4255C48
	Directory : datasets/lmd_matched/W/L/L
	Directory : datasets/lmd_matched/W/L/L/TRWLLOG128F425A988
	Directory : datasets/lmd_matched/W/L/L/TRWLLEC128F9326398
	Directory : datasets/lmd_matched/W/L/Y
	Directory : datasets/lmd_matched/W/L/Y/TRWLYAZ128F14776B4
	Directory : datasets/lmd_matched/W/L/W
	Directory : datasets/lmd_matched/W/L/W/TRWLWGT128F92D5AB4
	Directory : datasets/lmd_matched/W/Y
	Directory : datasets/lmd_matched/W/Y/R
	Directory : datasets/lmd_matched/W/Y/R/TRWYREX128F42B

	file 20470 not valid, skipping..
	Directory : datasets/lmd_matched/W/W/A
	Directory : datasets/lmd_matched/W/W/A/TRWWAVI128F42B79A1
	Directory : datasets/lmd_matched/W/W/F
	Directory : datasets/lmd_matched/W/W/F/TRWWFMG128E0793726
	Directory : datasets/lmd_matched/W/W/F/TRWWFQS12903CC22BB
	Directory : datasets/lmd_matched/W/W/F/TRWWFXP128F425339E
	Directory : datasets/lmd_matched/W/W/F/TRWWFNZ128E0788583
	Directory : datasets/lmd_matched/W/W/F/TRWWFAW12903CBEBB4
	Directory : datasets/lmd_matched/W/W/F/TRWWFFH12903CE12D2
	Directory : datasets/lmd_matched/W/W/O
	Directory : datasets/lmd_matched/W/W/O/TRWWOSE128F426D49F
	Directory : datasets/lmd_matched/W/W/O/TRWWOJJ128F424E301
	Directory : datasets/lmd_matched/W/W/O/TRWWOQO128F1499648
	Directory : datasets/lmd_matched/W/W/H
	Directory : datasets/lmd_matched/W/W/H/TRWWHPU128E078585C
	Directory : datasets/lmd_matched/W/W/M
	Directory : datasets/lmd_matched/W/W/M/TRWWMOF128F146CAE9
	Directory : datasets/lmd_matched/W/W/M/TRWWMWX128F428686C

In [ ]:
# archive:
"""
total: 116'189

part 1 : 26'000+   157.7 MB
part 2 : 21'000    157.1 MB
part 3 : 21'000    135.3 MB
part 4 : 25'000    141.5 MB
part 5 : 20'612
...

"""

In [10]:
path='datasets/TM_dataset_enc9_part5'
np.savez_compressed(
    path,
    array1_drm=array_drum,
    array2_pia=array_piano,
    array3_gtr=array_gtr,
    array4_bss=array_bass)